# PyMC4 developer guide

PyMC4 is based on TensorFlow Probability. This provides many benefits including a rich library of probability distributions and inference algorithms. In order to use these inference algorithms, we must provide a tensor-in-tensor-out logp function. The input tensors are placeholder tensors on which a value for a random variable (RV) can be set, and the output is a tensor of the logp of the model evaluated of these inputs. During sampling, for example, the sampler would propose different input values by updating the input tensors and compute the corresponding model logp and gradients of the logp (which is provided via autodiff of the output tensor from the TensorFlow library).

This provides a challenge for PyMC4 because in order to know which input tensors to construct, we need to evaluate the users' model. For example, let's consider the (non-sensical) model:

```python
x = pm4.Normal(0, 1)
y = pm4.Normal(x**2, 1)
```

This model should familiar to anyone who knows `PyMC3` or `STAN`. However, it hides some subtle complexity. We need to keep track of two things simultaneously here. `x` and `y` as RVs as well as the log probabilities  of `x` and `y` evaluated over some input values. That is why in TensorFlow probability you need to write your model twice: once for its RVs, and once for its logps.

As `PyMC4` is supposed to focus on UX we want to hide this complexity from users. This, however, creates a chicken-and-egg type of problem: in order to create the tensor-in-tensor-out log probability function, we need to replace the RVs in the user's model with the input-tensors. But without having run the model, we don't know which input-tensors we need to create in the first place.

The approach we took is to have the user define a decorated function which we call a **model template**. Template, because it's not the model itself, but rather, the instruction of how to build a model. This template function gets called **twice** under different contexts which alter how an RV gets converted to a tensor:
1. Under `ForwardContext`: This collects the RV names, the RVs get converted to sample tensors which allow forward sampling.
2. Under `InferenceContext`: Now when an RV gets converted to a tensor, it gets replaced with the input-tensor from the log_prob function. When we compute the log_prob for an individual RV, we evaluate it over that tensor.

In [61]:
import pymc4 as pm
import tensorflow as tf
from scipy import stats

In [115]:
x = pm.Normal(mu=0, sigma=1, name='x')

So far this is still a `PyMC4` RV object:

In [116]:
x

When used in conjuction with TF, `.as_tensor()` gets called. By default, things are evaluated under the `ForwardContext`:

In [117]:
x.as_tensor()

<tf.Tensor: id=3445, shape=(), dtype=float32, numpy=0.6061011>

This is done automatically by TF by registering a `tensor_conversion_function` (in `__init__.py`):

In [118]:
1 * x

<tf.Tensor: id=3449, shape=(), dtype=float32, numpy=0.6061011>

In [122]:
# Value sampled from a normal distribution
sample = x.as_tensor().numpy()
sample

0.6061011

Once we require the logp, we call `.log_prob()` which evaluates the distribution *over itself*:

In [123]:
x.log_prob()

<tf.Tensor: id=3512, shape=(), dtype=float32, numpy=-1.1026177>

In [124]:
stats.norm().logpdf(sample)

-1.1026178022947524

In the context of a model, we can see what is going on more clearly by doing things manually:

In [125]:
@pm.model
def model_template():
    print('model is called')
    x = pm.Normal(mu=0, sigma=1, name='x')
    print('x is type', x)
    print('x viewed as a tensor is', x.as_tensor())
    print('x**2 is', x**2)

In [126]:
model = model_template.configure()

model is called
x is type <pymc4.random_variables.continuous.Normal object at 0x1a28f4c7f0>
x viewed as a tensor is tf.Tensor(-0.68766874, shape=(), dtype=float32)
x**2 is tf.Tensor(0.4728883, shape=(), dtype=float32)


In [127]:
# So far, only the forward context is created with the registered RVs.
model._forward_context.vars

In [128]:
with model._forward_context:
    print(model._forward_context.vars[0].as_tensor())

tf.Tensor(-0.68766874, shape=(), dtype=float32)


In [129]:
# Create our tensor-in tensor-out logp function
logp_func = model.make_log_prob_function()

In [131]:
x = tf.ones((10,)) * 2
logp_tensor = logp_func(x)

model is called
x is type <pymc4.random_variables.continuous.Normal object at 0x1a28f31f98>
x viewed as a tensor is tf.Tensor([2. 2. 2. 2. 2. 2. 2. 2. 2. 2.], shape=(10,), dtype=float32)
x**2 is tf.Tensor([4. 4. 4. 4. 4. 4. 4. 4. 4. 4.], shape=(10,), dtype=float32)


Something important has happened the second time the model template was called. Instead of the `RV` when converted to a tensor becoming a tensor-value sampled from a normal distribution as the first time, it now converts to the input-tensor. 

In a second step, we loop over the created RVs, compute their log probability tensors, and sum them:

In [132]:
logp_tensor

<tf.Tensor: id=3695, shape=(), dtype=float32, numpy=-29.189386>

This happens inside of `pymc4.Model.make_log_prob_function()`:

```python
def make_log_prob_function(self):
    """Return the log probability of the model."""

    def log_prob(*args):
        # Create the new context with the RVs
        # collected from the first pass where
        # it was stored in `self._forward_context`,
        # and link them to the input tensors provided 
        # by args.
        context = contexts.InferenceContext(
            args, 
            expected_vars=self._forward_context.vars)
        with context:
            # Call the model-template again, this time
            # RVs will evaluate to the input-tensors
            self._evaluate()
            # `var.log_prob()` will evaluate the
            # distribution's logp over the input-tensor
            return sum(tf.reduce_sum(var.log_prob()) for var in context.vars)

    # Return our tensor-in-tensor-out function
    return log_prob
```